In [ ]:
import speech_recognition as sr
from deep_translator import GoogleTranslator
import requests
from PIL import Image
from io import BytesIO
import speech_recognition as sr

recognizer = sr.Recognizer()

with sr.Microphone() as source:
    print("🎙️ Speak something...")
    audio_data = recognizer.listen(source)
    print("🔊 Recording done, saving file...")

    with open("sample_audio.wav", "wb") as f:
        f.write(audio_data.get_wav_data())
# MonsterAPI key
MONSTER_API_KEY = "wyJSspsJQbQ++oMEg0ILCvcGvfFlgvTcDy8TRudNows="
headers = {
    "Authorization": f"Bearer {MONSTER_API_KEY}",
    "Content-Type": "application/json"
}

def transcribe_speech(audio_file_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file_path) as source:
        audio = recognizer.record(source)

    try:
        # Detect language automatically using Google
        text = recognizer.recognize_google(audio, show_all=False)
        print("Detected speech:", text)
        return text
    except sr.UnknownValueError:
        print("Could not understand the audio.")
    except sr.RequestError as e:
        print("Error with the speech recognition service:", e)

def translate_to_english(text):
    try:
        translated = GoogleTranslator(source='auto', target='en').translate(text)
        print("Translated to English:", translated)
        return translated
    except Exception as e:
        print("Translation error:", e)
        return text

def generate_image_with_monsterapi(prompt):
    url = "https://api.monsterapi.ai/v1/generate"
    headers = {
        "Authorization": f"Bearer {MONSTER_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "stable-diffusion",
        "prompt": prompt,
        "num_images": 1,
        "width": 512,
        "height": 512
    }

    response = requests.post(url, json=payload, headers=headers)
    data = response.json()
    if 'data' in data and data['data']:
        image_url = data['data'][0]['url']
        img_response = requests.get(image_url)
        img = Image.open(BytesIO(img_response.content))
        img.show()
    else:
        print("Image generation failed:", data)

# Example usage:
audio_path = "sample_audio.wav"  # Replace with your audio file path
raw_text = transcribe_speech(audio_path)

if raw_text:
    english_prompt = translate_to_english(raw_text)
    generate_image_with_monsterapi(english_prompt)

🎙️ Speak something...
🔊 Recording done, saving file...
Detected speech: get me an image of a cat
Translated to English: get me an image of a cat
Image generation failed: {'message': "Authorization header requires 'Credential' parameter. Authorization header requires 'Signature' parameter. Authorization header requires 'SignedHeaders' parameter. Authorization header requires existence of either a 'X-Amz-Date' or a 'Date' header. (Hashed with SHA-256 and encoded with Base64) Authorization=8oMNfqTqSzdvvDg2ajiILAJ5KHhfcgctR+E6QoYNp0A="}
